<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP/DZ_4_QA_bert_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answering. Практическое задание (PJ)

Для закрепления материала модуля предлагаем вам решить задачу QA для датасета [SberQuad](https://huggingface.co/datasets/sberquad), используя любые доступные вам средства.

Для достижения наилучшего результата уделите внимание подбору гиперапарметров как в плане архитектуры, так и в плане обучения модели.

Критерии оценивания проекта:

- общее качество кода и следование PEP-8;
- использование рекуррентных сетей;
- использованы варианты архитектур, близкие к state of the art для данной задачи;
- произведен подбор гиперпараметров;
- использованы техники изменения learning rate (lr scheduler);
- использована адекватная задаче функция потерь;
- использованы техники регуляризации;
- корректно проведена валидация модели;
- использованы техники ensemble;
- использованы дополнительные данные;
- итоговое значение метрики качества > 0.75 (f1).


#PIP

In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install accelerate -U
!pip install transformers[torch] -q
exit() #restart


<a class="anchor" id="section3"></a>
<h2 style="color:green;font-size: 2em;">Understanding Data Preprocessing required for Question-Answering System</h2>

In this section before getting to the training part, let us understand how we process train data and validation data.

```
Note: For training and demo we will use distil bert instead of bert as it has less parameters and so consume less memory.
```

In [1]:
import transformers
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import warnings
warnings.simplefilter("ignore")

#**Loading dataset**

In [2]:
from datasets import load_dataset
dataset = load_dataset("sberquad")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

We have about 87599 data points in train and 10570 in validation.
Lets see some sample

In [3]:
# to make text bold
s_bold = '\033[1m'
e_bold = '\033[0;0m'

print(s_bold + 'Train Data Sample.....' + e_bold)
train_data = dataset["train"]
for data in train_data:
    print(' ')
    print(s_bold + 'ID -' + e_bold, data['id'])
    print(s_bold +'TITLE - '+ e_bold, data['title'])
    print(s_bold + 'CONTEXT - '+ e_bold,data['context'])
    print(s_bold + 'ANSWERS - ' + e_bold,data['answers']['text'])
    print(s_bold + 'ANSWERS START INDEX - ' + e_bold,data['answers']['answer_start'])
    print(' ')
    break

print('---'*30)
print(s_bold + 'Validation Data Sample.....' + e_bold)
train_data = dataset["validation"]
for data in train_data:
    print(' ')
    print(s_bold + 'ID -' + e_bold, data['id'])
    print(s_bold +'TITLE - '+ e_bold, data['title'])
    print(s_bold + 'CONTEXT - '+ e_bold,data['context'])
    print(s_bold + 'ANSWERS - ' + e_bold,data['answers']['text'])
    print(s_bold + 'ANSWERS START INDEX - ' + e_bold,data['answers']['answer_start'])
    print(' ')
    break



Train Data Sample.....
 
ID - 62310
TITLE -  SberChallenge
CONTEXT -  В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
ANSWERS -  ['известковыми выделениями сине-зелёных водорослей']
ANSWERS START INDEX -  [109]
 
------------------------------------------------------------------------------------------
Validation Data Sample.....
 
ID - 60544
TITLE -  SberChallenge
CONTEXT -  Первые упоминания о строении человечес

Another important thing that we found is that we can see multiple answers in one of our validation sample.Lets analyze it further.

In [4]:
dataset["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/45328 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [5]:
dataset["validation"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/5036 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

# Take rand_part datasets

In [6]:
i =  2167
dataset["validation"][i] , dataset["test"][i]

({'id': 76422,
  'title': 'SberChallenge',
  'context': 'Относительно вхождения уральской семьи языков в более крупные генетические объединения существуют разные гипотезы, ни одна из которых не признана специалистами по уральским языкам. Согласно ностратической гипотезе, уральская семья, наряду с другими языковыми семьями и макросемьями входит в состав более крупного образования — ностратической макросемьи, причём сближается там с юкагирскими языками, образуя уральско-юкагирскую группу. Данная позиция, однако, была подвергнута критике различными специалистами, считается весьма спорной и её выводы не принимаются многими компаративистами, которые рассматривают теорию ностратических языков либо как, в худшем случае, полностью ошибочную или как, в лучшем случае, просто неубедительную[8][9]. Примерно до середины 1950-х гг. была популярна урало-алтайская гипотеза, объединявшая в одну макросемью уральские и алтайские языки, однако в настоящее время она не пользуется поддержкой лингвистов. С у

In [7]:
part_of_data = 0.1

set_ids = np.random.choice(range(len(dataset["validation"])), int(len(dataset["validation"])*part_of_data*2.2), replace=False)
len(set_ids)

1107

In [8]:
choice = np.random.choice(range(len(set_ids)), int(len(set_ids)*0.2), replace=False)
test_mask = np.zeros(set_ids.shape[0], dtype=bool)
test_mask[choice] = True

test_ids = set_ids[test_mask]
val_ids = set_ids[~test_mask]
len(val_ids), len(test_ids)

(886, 221)

In [9]:

from datasets import Dataset, DatasetDict


DATASET = DatasetDict({
    'train': dataset["train"].select(
            np.random.choice(range(len(dataset["train"])), int(len(dataset["train"])*part_of_data), replace=False)
        ),
    'validation': dataset["validation"].select(val_ids
        ),
    'test': dataset["validation"].select(test_ids
        )
})

DATASET

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4532
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 886
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 221
    })
})

# **Data Preprocessing**



**How to handle long contexts??**

Here we have relatively smaller context length. But what if its very large. Then during truncation there is a chance that answer might get truncated and context will miss the answer.To solve this problem we create several features of different pieces of context. The only thing we must aware is to add enough overlap between contexts.
This can be done by tokenizer itself.


In [10]:
from transformers import AutoTokenizer

trained_checkpoint = "timpal0l/mdeberta-v3-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

context = DATASET["train"][0]["context"]
question = DATASET["train"][0]["question"]
answer = DATASET["train"][0]["answers"]["text"]


inputs = tokenizer(
    question,
    context,
    max_length=160,
    truncation="only_second",  # only to truncate context
    stride=70,  # no of overlapping tokens  between concecute context pieces
    return_overflowing_tokens=True,  #to let tokenizer know we want overflow tokens
)


print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

print('Question: ',question)
print(' ')
print('Context : ',context)
print(' ')
print('Answer: ', answer)
print('--'*25)

for i, ids in enumerate(inputs["input_ids"]):
    print('Context piece', i+1)
    print(tokenizer.decode(ids[ids.index(1):]))
    print(' ')



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The 4 examples gave 3 features.
Here is where each comes from: [0, 0, 0].
Question:  Какие территории входили в Тамбовское наместничество?
 
Context :  Тамбовское наместничество было существенно больше современной Тамбовской области: так, туда входили часть современной Мордовии, Липецкой, Воронежской, Пензенской и Рязанской областей. Внешне Тамбов напоминал тогда большое село: те же деревянные дома, соломенные в большинстве крыши, улицы без мостовых, огороды. И только крепость говорила, что это город. Таким он был до конца последней четверти XVIII века. После того, как город получил статус административного центра губернии, он стал перестраиваться. Столичные архитекторы составили план застройки Тамбова. По этому плану развернулось строительство ряда крупных зданий, в их числе был построен гостиный двор (ныне здание универмага). В это же время принимались и другие меры, подсказанные новым значением города и духом времени.
 
Answer:  ['часть современной Мордовии, Липецкой, Воронежской, П

Actually Hugging face will take care of these. We only need to pass Start index and End index corresponding to each input. If no answer is present in context we need to pass start and end position as 0.

In [11]:
def train_data_preprocess(examples):

    """
    generate start and end indexes of answer in context
    """

    def find_context_start_end_index(sequence_ids):
        """
        returns the token index in whih context starts and ends
        """
        token_idx = 0
        while sequence_ids[token_idx] != 1:  #means its special tokens or tokens of queston
            token_idx += 1                   # loop only break when context starts in tokens
        context_start_idx = token_idx

        while sequence_ids[token_idx] == 1:
            token_idx += 1
        context_end_idx = token_idx - 1
        return context_start_idx,context_end_idx


    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    answers = examples["answers"]

    inputs = tokenizer(
        questions,
        context,
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,  #returns id of base context
        return_offsets_mapping=True,  # returns (start_index,end_index) of each token
        padding="max_length"
    )


    start_positions = []
    end_positions = []


    for i,mapping_idx_pairs in enumerate(inputs['offset_mapping']):
        context_idx = inputs['overflow_to_sample_mapping'][i]

        # from main context
        answer = answers[context_idx]
        answer_start_char_idx = answer['answer_start'][0]
        answer_end_char_idx = answer_start_char_idx + len(answer['text'][0])


        # now we have to find it in sub contexts
        tokens = inputs['input_ids'][i]
        sequence_ids = inputs.sequence_ids(i)

        # finding the context start and end indexes wrt sub context tokens
        context_start_idx,context_end_idx = find_context_start_end_index(sequence_ids)

        #if the answer is not fully inside context label it as (0,0)
        # starting and end index of charecter of full context text
        context_start_char_index = mapping_idx_pairs[context_start_idx][0]
        context_end_char_index = mapping_idx_pairs[context_end_idx][1]


        #If the answer is not fully inside the context, label is (0, 0)
        if (context_start_char_index > answer_start_char_idx) or (
            context_end_char_index < answer_end_char_idx):
            start_positions.append(0)
            end_positions.append(0)

        else:

            # else its start and end token positions
            # here idx indicates index of token
            idx = context_start_idx
            while idx <= context_end_idx and mapping_idx_pairs[idx][0] <= answer_start_char_idx:
                idx += 1
            start_positions.append(idx - 1)


            idx = context_end_idx
            while idx >= context_start_idx and mapping_idx_pairs[idx][1] > answer_end_char_idx:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_sample = DATASET["train"].select([i for i in range(200)])

train_dataset = train_sample.map(
    train_data_preprocess,
    batched=True,
    remove_columns=DATASET["train"].column_names
)

len(DATASET["train"]),len(train_dataset)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

(4532, 203)

We can see a increase in number of datapoints after the tokenization method we used.Lets compare the values before and after tokenization.We will print some of the questions,context and answers after tokenization and compare with the original one.

In [12]:
def print_context_and_answer(idx,mini_ds=dataset["train"]):

    print(idx)
    print('----')
    question = mini_ds[idx]['question']
    context = mini_ds[idx]['context']
    answer = mini_ds[idx]['answers']['text']
    print('Theoretical values :')
    print(' ')
    print('Question: ')
    print(question)
    print(' ')
    print('Context: ')
    print(context)
    print(' ')
    print('Answer: ')
    print(answer)
    print(' ')
    answer_start_char_idx = mini_ds[idx]['answers']['answer_start'][0]
    answer_end_char_idx = answer_start_char_idx + len(mini_ds[idx]['answers']['text'][0])
    print('Start and end index of text: ',answer_start_char_idx,answer_end_char_idx)
    print('----'*20)
    print('Values after tokenization:')


    #answer
    sep_tok_index = train_dataset[idx]['input_ids'].index(1) #get index for [SEP]
    question_ = train_dataset[idx]['input_ids'][:sep_tok_index+1]
    question_decoded = tokenizer.decode(question_)
    context_ = train_dataset[idx]['input_ids'][sep_tok_index+1:]
    context_decoded = tokenizer.decode(context_)
    start_idx = train_dataset[idx]['start_positions']
    end_idx = train_dataset[idx]['end_positions']
    answer_toks = train_dataset[idx]['input_ids'][start_idx:end_idx]
    answer_decoded = tokenizer.decode(answer_toks)
    print(' ')
    print('Question: ')
    print(question_decoded)
    print(' ')
    print('Context: ')
    print(context_decoded)
    print(' ')
    print('Answer: ')
    print(answer_decoded)
    print(' ')
    print('Start pos and end pos of tokens: ',train_dataset[idx]['start_positions'],train_dataset[idx]['end_positions'])
    print('____'*20)


print_context_and_answer(0)
print_context_and_answer(1)
print_context_and_answer(2)
print_context_and_answer(3)

0
----
Theoretical values :
 
Question: 
чем представлены органические остатки?
 
Context: 
В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
 
Answer: 
['известковыми выделениями сине-зелёных водорослей']
 
Start and end index of text:  109 157
--------------------------------------------------------------------------------
Values after tokenization:
 
Question: 
[CLS]
 
Context: 
Какие территории входили в Тамбовс

# Metrics needed for Evaluation

How to evaluate the model?

Lets take a small eval set.Here we donot need to do much preprocesing. . We will use the pretrained "distilbert-base-uncased" weights which is not fine tuned and lets see how the model performs.

- We will set offset to None for all those questions part of the data.
- We will also append base context id to each sample
- We will evalue using our untuned bert-base model and lets see the performance.

In [13]:
def preprocess_validation_examples(examples):
    """
    preprocessing validation data
    """
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")

    base_ids = []

    for i in range(len(inputs["input_ids"])):

        # take the base id (ie in cases of overflow happens we get base id)
        base_context_idx = sample_map[i]
        base_ids.append(examples["id"][base_context_idx])

        # sequence id indicates the input. 0 for first input and 1 for second input
        # and None for special tokens by default
        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        # for Question tokens provide offset_mapping as None
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["base_id"] = base_ids
    return inputs


data_val_sample = DATASET["validation"].select([i for i in range(100)])
eval_set = data_val_sample.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=DATASET["validation"].column_names,
)
len(eval_set)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100

In [15]:
import torch
from transformers import AutoModelForQuestionAnswering


# take a small sample
eval_set_for_model = eval_set.remove_columns(["base_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}

model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)


with torch.no_grad():
    outputs = model(**batch)

start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

start_logits.shape,end_logits.shape

((100, 512), (100, 512))

In [16]:
import numpy as np
import collections
import evaluate

def predict_answers_and_evaluate(start_logits,end_logits,eval_set,examples):
    """
    make predictions
    Args:
    start_logits : strat_position prediction logits
    end_logits: end_position prediction logits
    eval_set: processed val data
    examples: unprocessed val data with context text
    """
    # appending all id's corresponding to the base context id
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(eval_set):
        example_to_features[feature["base_id"]].append(idx)

    n_best = 20
    max_answer_length = 30
    predicted_answers = []

    for example in examples:
        example_id = example["id"]
        context = example["context"]
        answers = []

        # looping through each sub contexts corresponding to a context and finding
        # answers
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = eval_set["offset_mapping"][feature_index]

            # sorting the predictions of all hidden states and taking best n_best prediction
            # means taking the index of top 20 tokens
            start_indexes = np.argsort(start_logit).tolist()[::-1][:n_best]
            end_indexes = np.argsort(end_logit).tolist()[::-1][:n_best]


            for start_index in start_indexes:
                for end_index in end_indexes:

                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                       ):
                        continue

                    answers.append({
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                        })


            # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    metric = evaluate.load("squad")

    theoretical_answers = [
            {"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples
    ]

    metric_ = metric.compute(predictions=predicted_answers,
                             references=theoretical_answers)
    return predicted_answers, metric_



Let us evaluate the model.This metric expects the predicted answers in the format we saw above (a list of dictionaries with one key for the ID of the example and one key for the predicted text) and the theoretical answers in the format.

In [17]:
pred_answers, metrics_ = predict_answers_and_evaluate(start_logits, end_logits,
                                                     eval_set, data_val_sample)
metrics_

{'exact_match': 49.0, 'f1': 79.94628704628703}

We have very poor score as expected. Now we will fine tune the model and will see the performance in whole validation dataset.

#Training a Question Answering System based on Bert

Let us define a Pyorch dataloader

In [18]:
from torch.utils.data import DataLoader, Dataset


class DataQA(Dataset):
    def __init__(self, dataset, mode="train"):
        self.mode = mode


        if self.mode == "train":
            # sampling
            self.dataset = dataset["train"]
            self.data = self.dataset.map(train_data_preprocess,
                                                      batched=True,
                            remove_columns= dataset["train"].column_names)

        elif self.mode == "validation":

            self.dataset = dataset["validation"]
            self.data = self.dataset.map(preprocess_validation_examples,
                                         batched=True,
                                         remove_columns = dataset["validation"].column_names,
               )
        else:
            self.dataset = dataset["test"]
            self.data = self.dataset.map(preprocess_validation_examples,
                                         batched=True,
                                         remove_columns = dataset["test"].column_names,
               )


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        out = {}
        example = self.data[idx]
        out['input_ids'] = torch.tensor(example['input_ids'])
        out['attention_mask'] = torch.tensor(example['attention_mask'])


        if self.mode == "train":

            out['start_positions'] = torch.unsqueeze(torch.tensor(example['start_positions']),dim=0)
            out['end_positions'] = torch.unsqueeze(torch.tensor(example['end_positions']),dim=0)

        return out


In [19]:
train_dataset = DataQA(DATASET, mode="train")
val_dataset = DataQA(DATASET, mode="validation")



for i,d in enumerate(train_dataset):
    for k in d.keys():
        print(k + ' : ', d[k].shape)
    print('--'*40)

    if i == 3:
        break

print('__'*50)

for i,d in enumerate(val_dataset):
    for k in d.keys():
        print(k + ' : ', len(d[k]))
    print('--'*40)

    if i == 3:
        break

Map:   0%|          | 0/4532 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
____________________________________________________________________________________________________
input_ids :  512
attention_mask :  

Let us load the data in batches

In [20]:
from transformers import default_data_collator
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=2,
)
eval_dataloader = DataLoader(
    val_dataset, collate_fn=default_data_collator, batch_size=2
)




for batch in train_dataloader:
   print(batch['input_ids'].shape)
   print(batch['attention_mask'].shape)
   print(batch['start_positions'].shape)
   print(batch['end_positions'].shape)
   break

print('---'*20)

for batch in eval_dataloader:
   print(batch['input_ids'].shape)
   print(batch['attention_mask'].shape)
   break

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 1])
torch.Size([2, 1])
------------------------------------------------------------
torch.Size([2, 512])
torch.Size([2, 512])


#**Define Model**

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Available device: {device}')

model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)
model = model.to(device)

Available device: cpu


#**Model Training**

In [22]:
from transformers import AdamW
from tqdm.notebook import tqdm
import datetime
import numpy as np
import collections
import evaluate

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 3

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
print(total_steps)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



6867


We need processed validation data at the time of evaluation to get offsets for each context

In [23]:
# we need processed validation data to get offsets at the time of evaluation
validation_processed_dataset = DATASET["validation"].map(preprocess_validation_examples,
            batched=True,remove_columns = DATASET["validation"].column_names,
               )

In [24]:
import random,time
import numpy as np

# to reproduce results
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


#storing all training and validation stats
stats = []


#to measure total training time
total_train_time_start = time.time()

for epoch in range(epochs):
    print(' ')
    print(f'=====Epoch {epoch + 1}=====')
    print('Training....')

    # ===============================
    #    Train
    # ===============================
    # measure how long training epoch takes
    t0 = time.time()

    training_loss = 0
    # loop through train data
    model.train()
    for step, batch in enumerate(train_dataloader):

        # we will print train time in every 40 epochs
        if step%40 == 0 and not step == 0:
              elapsed_time = format_time(time.time() - t0)
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed_time))



        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)



        #set gradients to zero
        model.zero_grad()

        result = model(input_ids = input_ids,
                        attention_mask = attention_mask,
                        start_positions = start_positions,
                        end_positions = end_positions,
                        return_dict=True)

        loss = result.loss

        #accumulate the loss over batches so that we can calculate avg loss at the end
        training_loss += loss.item()

        #perform backward prorpogation
        loss.backward()

        # update the gradients
        optimizer.step()

    # calculate avg loss
    avg_train_loss = training_loss/len(train_dataloader)

    # calculates training time
    training_time = format_time(time.time() - t0)


    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    # ===============================
    #    Validation
    # ===============================

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()


    start_logits,end_logits = [],[]
    for step, batch in enumerate(eval_dataloader):


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)


        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
             result = model(input_ids = input_ids,
                        attention_mask = attention_mask,return_dict=True)



        start_logits.append(result.start_logits.cpu().numpy())
        end_logits.append(result.end_logits.cpu().numpy())


    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    # start_logits = start_logits[: len(val_dataset)]
    # end_logits = end_logits[: len(val_dataset)]




    # calculating metrics
    answers,metrics_ = predict_answers_and_evaluate(start_logits,
                                                    end_logits,
                                                    validation_processed_dataset,
                                                    DATASET["validation"])
    print(f'Exact match: {metrics_["exact_match"]}, F1 score: {metrics_["f1"]}')


    print('')
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_train_time_start)))


 
=====Epoch 1=====
Training....
  Batch    40  of  2,289.    Elapsed: 0:03:29.
  Batch    80  of  2,289.    Elapsed: 0:07:01.
  Batch   120  of  2,289.    Elapsed: 0:10:36.
  Batch   160  of  2,289.    Elapsed: 0:14:05.
  Batch   200  of  2,289.    Elapsed: 0:17:34.
  Batch   240  of  2,289.    Elapsed: 0:21:02.
  Batch   280  of  2,289.    Elapsed: 0:24:28.
  Batch   320  of  2,289.    Elapsed: 0:27:58.
  Batch   360  of  2,289.    Elapsed: 0:31:25.
  Batch   400  of  2,289.    Elapsed: 0:34:51.
  Batch   440  of  2,289.    Elapsed: 0:38:19.
  Batch   480  of  2,289.    Elapsed: 0:41:45.
  Batch   520  of  2,289.    Elapsed: 0:45:12.
  Batch   560  of  2,289.    Elapsed: 0:48:39.
  Batch   600  of  2,289.    Elapsed: 0:52:08.
  Batch   640  of  2,289.    Elapsed: 0:55:36.
  Batch   680  of  2,289.    Elapsed: 0:59:05.
  Batch   720  of  2,289.    Elapsed: 1:02:33.
  Batch   760  of  2,289.    Elapsed: 1:06:01.
  Batch   800  of  2,289.    Elapsed: 1:09:28.
  Batch   840  of  2,289.  

#**Model Testing**

In [25]:
test_dataset = DataQA(DATASET, mode="test")

test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=2
)


# we need processed test data to get offsets at the time of evaluation
test_processed_dataset = DATASET["test"].map(preprocess_validation_examples,
                                             batched=True,
                                             remove_columns = DATASET["test"].column_names,
               )

# during evaluation.
model.eval()


start_logits,end_logits = [],[]
for step, batch in enumerate(test_dataloader):


    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)


    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():
          result = model(input_ids = input_ids,
                    attention_mask = attention_mask,return_dict=True)



    start_logits.append(result.start_logits.cpu().numpy())
    end_logits.append(result.end_logits.cpu().numpy())


start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
# start_logits = start_logits[: len(val_dataset)]
# end_logits = end_logits[: len(val_dataset)]




# calculating metrics
answers,metrics_ = predict_answers_and_evaluate(start_logits,
                                                end_logits,
                                                test_processed_dataset,
                                                DATASET["test"])
print(f'Exact match: {metrics_["exact_match"]}, F1 score: {metrics_["f1"]}')



Map:   0%|          | 0/221 [00:00<?, ? examples/s]

Exact match: 43.89140271493213, F1 score: 78.07398721394351
